In [1]:
import cv2
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import pygame
import threading
import time

x_start, y_start, width, height = 100, 100, 200, 200  

# Email configuration
SMTP_SERVER = 'smtp.gmail.com'  # Gmail SMTP server address
SMTP_PORT = 587  # Gmail SMTP port
EMAIL = 'your_email@gmail.com'  # Replace with your email
PASSWORD = 'your_app_password'  # Replace with your generated app password
RECIPIENT_EMAIL = 'recipient_email@gmail.com'  # Replace with recipient email

# Cooldown period (in seconds)
COOLDOWN_PERIOD = 30
last_sent_time = 0

# Initialize pygame mixer
pygame.mixer.init()

def send_email():
    msg = MIMEMultipart()
    msg['From'] = EMAIL
    msg['To'] = RECIPIENT_EMAIL
    msg['Subject'] = 'Motion Detected'
    
    body = 'Motion is detected'
    msg.attach(MIMEText(body, 'plain'))
    
    try:
        server = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        server.starttls()
        server.login(EMAIL, PASSWORD)
        text = msg.as_string()
        server.sendmail(EMAIL, RECIPIENT_EMAIL, text)
        server.quit()
        print("Email sent successfully.")
    except Exception as e:
        print(f"Error: unable to send email. {e}")

def play_buzzer():
    if not pygame.mixer.music.get_busy():
        pygame.mixer.music.load('buzzer.mp3')
        pygame.mixer.music.play(-1)  # Play in a loop

def stop_buzzer():
    if pygame.mixer.music.get_busy():
        pygame.mixer.music.stop()

def main():
    global last_sent_time
    cap = cv2.VideoCapture(0) 
    if not cap.isOpened():
        print("Error: Could not open video stream.")
        return
    
    ret, frame1 = cap.read()
    
    if not ret:
        print("Error: Could not read frame.")
        return

    # Mirror the first frame
    frame1 = cv2.flip(frame1, 1)

    gray1 = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)
    gray1 = cv2.GaussianBlur(gray1, (21, 21), 0)

    while cap.isOpened():
        ret, frame2 = cap.read()
        if not ret:
            break

        # Mirror the second frame
        frame2 = cv2.flip(frame2, 1)

        gray2 = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
        gray2 = cv2.GaussianBlur(gray2, (21, 21), 0)

        diff = cv2.absdiff(gray1, gray2)
        _, thresh = cv2.threshold(diff, 25, 255, cv2.THRESH_BINARY)
        thresh = cv2.dilate(thresh, None, iterations=2)

        roi = thresh[y_start:y_start+height, x_start:x_start+width]

        contours, _ = cv2.findContours(roi, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        cv2.rectangle(frame2, (x_start, y_start), (x_start + width, y_start + height), (0, 255, 0), 2)

        if contours:
            for contour in contours:
                if cv2.contourArea(contour) < 500:
                    continue

                (x, y, w, h) = cv2.boundingRect(contour)
                cv2.rectangle(frame2, (x_start + x, y_start + y), (x_start + x + w, y_start + y + h), (0, 0, 255), 2)

                current_time = time.time()
                if current_time - last_sent_time > COOLDOWN_PERIOD:
                    last_sent_time = current_time
                    # Send email when motion is detected
                    send_email()
                
                # Play buzzer when motion is detected
                threading.Thread(target=play_buzzer).start()
        else:
            # Stop buzzer when no motion is detected
            stop_buzzer()

        cv2.imshow("Frame", frame2)

        gray1 = gray2

        if cv2.waitKey(30) & 0xFF == 27:  # press "esc" to exit
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


pygame 2.6.0 (SDL 2.28.4, Python 3.12.4)
Hello from the pygame community. https://www.pygame.org/contribute.html
Error: unable to send email. (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials 98e67ed59e1d1-2cfdc4064cesm8305757a91.12 - gsmtp')
Error: unable to send email. (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials d9443c01a7336-1ff5905f9b9sm47966955ad.130 - gsmtp')
Error: unable to send email. (535, b'5.7.8 Username and Password not accepted. For more information, go to\n5.7.8  https://support.google.com/mail/?p=BadCredentials d9443c01a7336-1ff59295371sm48272925ad.257 - gsmtp')
